# Information Theory and Wordle 2/2 (Julia)

### Overview
So far we've defined information theory, the entropy function and the parameters of the game Wordle.

Now the task is to find the best first guess in the game, that is the word with the highest entropy. 

### Data
There are two text files - one with the possible words that appear as answers in the game (which can be found by inspecting the wordle page) and one with a list of possible words that are accepted as guesses. 

In [1]:
# load list of possible answers (taken from the game's website)
data = open("possible_words.txt", "r")
answers = readlines(data)

# load list of possible guesses (valid five letter words that wordle will accept)
data2 = open("allowed_guesses.txt", "r")
guesses = readlines(data2)

12972-element Vector{String}:
 "aahed"
 "aalii"
 "aargh"
 "aarti"
 "abaca"
 "abaci"
 "aback"
 "abacs"
 "abaft"
 "abaka"
 "abamp"
 "aband"
 "abase"
 ⋮
 "zowee"
 "zowie"
 "zulus"
 "zupan"
 "zupas"
 "zuppa"
 "zurfs"
 "zuzim"
 "zygal"
 "zygon"
 "zymes"
 "zymic"

In [2]:
length(answers), length(guesses) 

(2315, 12972)

There are 2,315 possible words that can be the Wordle solution. However, the game accepts 12,972 different five letter words (many of them being somewhat strange such as "aahed"). The task is to find which word among the list of words in 'guesses' will do the best job at reducing the outcome space since the smaller the outcome space the easier it is to guess the correct answer. Even by eyeballing the output of the list, it's clear that some words are better than others "abase" seems better than "zuppa."

If we can apply the entropy function to "abase" and "zuppa" and compare the number of bits each word gives, then we will have a numeric answer to this natural guess. In order to apply the entropy function to a given guess word, we need some sort of measurement of probability that the guess is the answer.

One way to do this would be: for each of the 12,972 possible guesses, loop through all 2,315 possible answers compute the coloring vector as done in the previous notebook (represent each guess as a vector of 0's, 1's and 2's s.t. 0=grey, 1=yellow, 2=green) and compute the entropy of the distribution obtained by counting all the matches for a given guess then selecting which guess out of all guesses has the highest entropy. 

However, due to the length of the word sets, this approach is too time consuming so a more efficient method involves a 'pre-processing step' in which we loop over the solution set of words, and create a dictionary with the letters of the alphabet as keys, and the values of each key(letter) being a vector of 5 digits representing the number of times the letter occurs in the first, second, third, fourth and fifth position. What this allows us to do is get a rough estimate of what structure the best guess should have, meaning what letters are best in which position. 

### Making Letter Frequency Dictionary

In [3]:
# loops over the answer list and counts the number of times each letter appears in each position
letter_counts = Dict()
for key in ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    letter_counts[key] = [0,0,0,0,0]
    for i in 1:5
        for word in answers
            if word[i]==key
                letter_counts[key][i]+=1
            end
        end
    end  
end
letter_counts

Dict{Any, Any} with 26 entries:
  'n' => [37, 87, 139, 182, 130]
  'f' => [136, 8, 25, 35, 26]
  'w' => [83, 44, 26, 25, 17]
  'd' => [111, 20, 75, 69, 118]
  'e' => [72, 242, 177, 318, 424]
  'o' => [41, 279, 244, 132, 58]
  'h' => [69, 144, 9, 28, 139]
  'j' => [20, 2, 3, 2, 0]
  'i' => [34, 202, 266, 158, 11]
  'k' => [20, 10, 12, 55, 113]
  'r' => [105, 267, 163, 152, 212]
  's' => [366, 16, 80, 171, 36]
  't' => [149, 77, 111, 139, 253]
  'q' => [23, 5, 1, 0, 0]
  'y' => [6, 23, 29, 3, 364]
  'a' => [141, 304, 307, 163, 64]
  'c' => [198, 40, 56, 152, 31]
  'p' => [142, 61, 58, 50, 56]
  'm' => [107, 38, 61, 68, 42]
  'z' => [3, 2, 11, 20, 4]
  'g' => [115, 12, 67, 76, 41]
  'v' => [43, 15, 49, 46, 0]
  'l' => [88, 201, 112, 162, 156]
  'u' => [33, 186, 165, 82, 1]
  'x' => [0, 14, 12, 3, 8]
  'b' => [173, 16, 57, 24, 11]

We can get a good idea for what kind of word appears most often as one of the wordle answers by selecting the letter that has the highest value in each of the five places. For example, the highest value in the first position is 366 which belongs to 's'. The highest value in the second position is '304' belonging to 'a'. The highest value in the third position is '307' belonging to 'a'. And '318' corresponding to 'e' for the fourth position and '424' corresponding to 'e' again in the fifth position. All together that is "SAAEE" which is not a word, but does tell us that S is a good starting letter and A in the second or third position and E in the fourth or fifth position is likely to be close to the answer. 

This dictionary also tells us what kinda of words are least likely to be the answer since none of the answers contained 'x' in the first position or 'j' or 'q' in the last position.

### Computing Entropy of a Word Guess
Let's test the hypothesis that 'abase' is a better word than 'zuppa' to use as a first guess. 

For a given word, we can compute its entropy by getting a distributions of the greens, yellows and greys 


In [4]:
using StatsBase


In [0]:
entropy?

Computing the entropy of a given word is the hardest part of the task. We need to get a distribution of [greens, yellows, greys] for each individual letter then compute the entropy of the letter, then add up the  5 entropies of the five letters of the word. 

- First to get the 'greens' component of the distribution of the letter we just need to take the value in the dictionary corresponding to the index of that letter. 
- Next, the 'yellows' are obtained by taking the sum of the remaining values in the four other indices for that letter.
- Lastly the 'greys' are just the number of words - yellows - greens

Storing these three values for each letter gives a 3-element vector which is the distribution for that letter. The distribution becomes a probability by dividing each element by the total sum of the greens+yellows+greys. Finally, the entropy of that probability vector can be computed. 

In [7]:
function entropy_of_word(word, wordset)
    H = 0
    for (i,char) in enumerate(word)
        greens = letter_counts[char][i]
        yellows = sum(letter_counts[char]) - greens
        greys = length(wordset) - yellows - greens
        dist = [greens, yellows, greys]
        dist = dist ./sum(dist)
        H += entropy(dist)
        end
   return  H
end

entropy_of_word (generic function with 1 method)

Comparing the entropy (# of bits gained) of "abase" vs "zuppa"

In [8]:
entropy_of_word("abase", guesses)

1.3300237195818254

In [9]:
entropy_of_word("zuppa", guesses)

0.7678557106567846

So as predicted,"abase" is a better guess than "zuppa" since it has more bits - meaning it does a better job at narrowing down the remaining words that the answer could be.

### Finding the Word with Maximum Entropy

In [10]:
function find_best_word(wordset)
    best_word = ""
    best_entropy = 0
    for word in wordset
        H = 0
        H += entropy_of_word(word, wordset)
        if H > best_entropy
            best_entropy = H
            best_word = word
        end
    end
    return(best_word, best_entropy)
    end

find_best_word (generic function with 1 method)

## Results

Let's see how the function compares when we run it on the answer set vs the allowed guess set

In [11]:
#searching for highest entropy among the 12,972 possible guesses
find_best_word(guesses)

("areae", 1.622623743402735)

In [12]:
#searching for highest entropy among the 2,314 possible answers
find_best_word(answers)

("tease", 4.480724406556044)

From the results "tease" is clearly the better word to use as a first guess since it's entropy is over three times as high and it also contains more distinct letters. However the best guess still doesn't seem entirely correct since ideally the best first guess would contain 5 distinct letters and tease only contains four distinct letters. 

There are a couple explanations for these results.
1. Going back to the 'simplified' model created - it does not take into account double letters exactly like Wordle does. Mainly, if a letter in a guess is repeated n times but occurs m times in the actual answer with m<n, then Wordle will only color the guess with green/yellow up to m times. I did not create edge cases in the entropy function to handle certain situations which occur when there are repeated letters - but I linked two references that handled those situations.
2. The results are almost always better when a model is 'trained' on the 'test' set. Meaning, since I fed the function `find_best_word` the answer list, it's not surprising that it returned a better word than when the possible guess list was used. 

## Solution Recap 

Information theory and the entropy function can be used to find the word with maximum entropy by the following steps:

1. Iterate over the possible answers and count the number of times each letter of the alphabet shows up in the first, second, third, fourth and fifth position
2. With these counts, we can get an estimate for the probability of getting a green, yellow and grey match for the letters a given guess and store it in a three element vector to represent the distribution
3. Compute the entropy of the distribution of green, yellow and grey for each letter and sum the entropies for the five letters of the word
4. The word among the guesses with the highest entropy is the best word guess

## Additional Resources
Information theory has other applications to different areas of cryptography, computer science and physics. 

There are a lot of different implementations online of Wordle solvers using information theory - interestingly most find different 'best' words
 - https://aditya-sengupta.github.io/coding/2022/01/13/wordle.html (explains the edge case with repeated letters)
 - https://www.youtube.com/watch?v=fRed0Xmc2Wg&t=1s (explains the edge case with repeated letters)
 - https://betterprogramming.pub/forget-luck-optimized-wordle-strategy-using-bigquery-c676771e316f (uses Google BigQuery)
